# 1. FEATURE BLOCK

In [2]:
import numpy
import torch
import torchvision
from torchvision import transforms

## 1.1 Import dataset and pretrained model

In [ ]:
# 数据集路径请按需更改
img_data = torchvision.datasets.ImageFolder("../dataset/caltech256/256_ObjectCategories",
                                    transform=transforms.Compose([
                                                transforms.Resize(256),
                                                transforms.CenterCrop(224),
                                                transforms.ToTensor()])
                                                )
data_loader = torch.utils.data.DataLoader(img_data, batch_size=1, shuffle=True)
net = torchvision.models.resnet18(pretrained=True)
net.fc = torch.nn.Sequential(net.fc, torch.nn.Softmax(dim=1))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device : "+str(device))

## 1.2 Get and save feature

In [ ]:
features = []
count = 0
total = 0
print("total data : " + str(len(data_loader)))

# 使用cpu
if(str(device) == 'cpu'):
    net.eval()
    for x, y in data_loader:
        with torch.no_grad():
            pred = net(x)
            pred = torch.cat((pred[0],),0)
            features.append(pred.numpy())
            count +=1
            if(count==500):
                total += count
                count = 0
                print("checkpoint : " + str(total))
# 使用gpu
elif(str(device) == 'cuda'):
    net = net.cuda()
    net.eval()
    for x, y in data_loader:
        with torch.no_grad():
            pred = net(x.cuda())
            pred = torch.cat((pred[0],y.cuda()),0)
            features.append(pred.cpu().numpy()) # 使用gpu
            count +=1
            if(count==500):
                total += count
                count = 0
                print("checkpoint : " + str(total))
   
numpy.savetxt("../feature/resnet18-imagenet2caltech256-features.csv",features,delimiter=',')


# 2. MODEL BLOCK

In [ ]:
import joblib
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## 2.1 Import features

In [ ]:
feature_data = numpy.loadtxt("../feature/resnet18-imagenet2caltech256-features.csv", delimiter=',')
X = feature_data[:,:-1]
y = feature_data[:,-1]
print("labels : ")
print(y)

## 2.2 Train and save model

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=42) # Answer of UNIVERSE
clf = svm.SVC(kernel='rbf',verbose=True) # default : C=1, gamma=1/NUM_OF_FEATURES
clf.fit(X_train,y_train)
joblib.dump(clf, "../model/resnet18-imagenet2caltech256-model.pkl")
# clf = joblib.load(clf, "../model/resnet18-imagenet2caltech256-model.pkl")

## 2.3 Test model

In [ ]:
y_hat = clf.predict(X_test)
acc = accuracy_score(y_test, y_hat)
print("accuary : " + str(acc))